In [1]:
from constants import *
from data_loading import *
from data_cleaning import *
from utils import *
from embeddings import *
from prediction import *

#import warnings
#warnings.filterwarnings("ignore", category=DeprecationWarning)


# load training set as DataFrame
train = load_df(TRAIN_NEG, TRAIN_POS, TWEET, LABEL, LABEL_NEG, LABEL_POS)

# load testing set as DataFrame
test = pd.DataFrame({TWEET: load_txt(TEST_DATA)})

# set patterns to remove, replace, and replace with
to_remove = "<user>"
to_replace = "[^a-zA-Z#]"
replace_value = " "

In [2]:
# clean training set
train = clean(train, TWEET, CLEAN_TWEET, to_remove, to_replace, replace_value)

#clean test set
test = clean(test, TWEET, CLEAN_TWEET, to_remove, to_replace, replace_value)

In [3]:
train.head()

,tweet,label,clean_tweet
id,,,
0,vinco tresorpack 6 ( difficulty 10 of 10 objec...,0,vinco tresorpack difficulti object disassembl ...
1,glad i dot have taks tomorrow ! ! #thankful #s...,0,glad dot tak tomorrow #thank #startho
2,1-3 vs celtics in the regular season = were fu...,0,vs celtic regular season fuck play playoff
3,<user> i could actually kill that girl i'm so ...,0,could actual kill girl sorri
4,<user> <user> <user> i find that very hard to ...,0,find hard believ im afraid


### Bag of Words

In [4]:
classify_bow(train, test, tweets_col = CLEAN_TWEET)

0.7608333333333334

In [5]:
classify_bow(train, test, tweets_col = TWEET)

0.76085

### TF IDF

In [6]:
classify_tfidf(train, test, tweets_col = CLEAN_TWEET)

0.7608

In [7]:
classify_tfidf(train, test, tweets_col = TWEET)

0.7618333333333334

### Fasttext (embedding)

In [8]:
#model = fasttext_model(train, model = "skipgram")
#print(model.words)
# TODO use embedding

### Fasttext (prediction)

#### on "clean_tweet"

In [ ]:
classify_fasttext(train, test, tweets_col = CLEAN_TWEET, max_iter = 50)

classifying: 40.0%, best_acc=0.780933333333333409333333333334

#### on "tweet"

In [ ]:
classify_fasttext(train, test, tweets_col = TWEET, max_iter = 50)

In [11]:
# TODO fix MemoryError 
#classifier2 = fasttext.load_model('models/word_deps_sg_500d/words500.npy')